# Flux and magnitude conversion

In this tutorial we will use the [F115W filter](http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?id=JWST/NIRCam.F115W&&mode=browse&gname=JWST&gname2=NIRCam#filter) of [JWST/NIRCam](https://science.nasa.gov/mission/webb/nircam/) to convert a magnitude into a flux and vice versa. The magnitudes are defined in the Vega System with the magnitude of Vega set to 0.03 for each filter (see [configuration file](https://species.readthedocs.io/en/latest/configuration.html)).

## Getting started

We start by importing `species` and initiating the database.

In [1]:
from species import SpeciesInit
from species.phot.syn_phot import SyntheticPhotometry

In [2]:
SpeciesInit()

species

Version: 0.10.3.dev11
Working folder: /Users/tomasstolker/applications/species/docs/tutorials

Creating species_config.ini... [DONE]
Creating species_database.hdf5... [DONE]
Creating data folder... [DONE]

Configuration settings:
   - Database: species_database.hdf5
   - Data folder: data
   - Magnitude of Vega: 0.03

Multiprocessing: mpi4py installed
Process number 1 out of 1...


## Magnitude to flux

We now create an instance of [SyntheticPhotometry](https://species.readthedocs.io/en/latest/species.phot.html#species.phot.syn_phot.SyntheticPhotometry) with the filter name as listed by the [SVO Filter Profile Service](http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?mode=browse&gname=JWST&gname2=NIRCam).

In [3]:
synphot = SyntheticPhotometry('JWST/NIRCam.F115W')

100%|████████████████████████████████████████| 288k/288k [00:00<00:00, 219MB/s]


Adding spectrum: Vega
Reference: Bohlin et al. 2014, PASP, 126
URL: https://ui.adsabs.harvard.edu/abs/2014PASP..126..711B


To covert from apparent magnitude to flux, we use the [magnitude_to_flux](https://species.readthedocs.io/en/latest/species.phot.html#species.phot.syn_phot.SyntheticPhotometry.magnitude_to_flux) method. The zero-point flux can be provided but is otherwise calculated from a flux-calibration Vega spectrum. Both the filter profile and the Vega spectrum will be downloaded and stored into the database. Let's calculate the flux for a magnitude of 15 +/- 0.2.

In [4]:
flux, error = synphot.magnitude_to_flux(15., error=0.2)
print(f'Flux (W m-2 um-1) = {flux:.2e} +/- {error:.2e}')

Flux (W m-2 um-1) = 4.04e-15 +/- 7.48e-16


## Flux to magnitude

To convert backwards from flux to magnitude, we use the [flux_to_magnitude](https://species.readthedocs.io/en/latest/species.phot.html#species.phot.syn_phot.SyntheticPhotometry.flux_to_magnitude) method. When the `parallax` (in mas) or `distance` (in pc) is provide, both the apparent and absolute magnitude are calculated. The uncertainty on the parallax/distance is propagated into the uncertainty of the absolute magnitude.

In [5]:
app_mag, abs_mag = synphot.flux_to_magnitude(flux, error=error, parallax=(10., 1.))
print(f'Apparent magnitude = {app_mag[0]:.2f} +/- {app_mag[1]:.2f}')
print(f'Absolute magnitude = {abs_mag[0]:.2f} +/- {abs_mag[1]:.2f}')

Apparent magnitude = 15.00 +/- 0.20
Absolute magnitude = 10.00 +/- 0.30


As expected, we obtain again the magnitude that we started out with.

## Zero point flux

Finally, let's see what the zero-point flux (i.e. the flux of Vega) is for the `JWST/NIRCam.F115W` filter. The value is stored by the `zero_point` attribute of the `SyntheticPhotometry` object.

In [6]:
print(f'Zero point flux (W m-2 um-1) = {synphot.zero_point:.5e}')

Zero point flux (W m-2 um-1) = 3.92908e-09


This flux is indeed very similar to the zero point that is provided on the website of the [SVO Filter Profile Service](http://svo2.cab.inta-csic.es/svo/theory/fps3/index.php?id=JWST/NIRCam.F115W&&mode=browse&gname=JWST&gname2=NIRCam#filter): 3.92904e-10 erg cm$^{-2}$ s$^{-1}$ A$^{-1}$.